<a href="https://colab.research.google.com/github/metehanunal0/Transportation-LP-/blob/main/Transportation_LP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**RANDOM NUMBER GENERATOR**#

In [ ]:
import pandas as pd
import numpy as np
import math

# the seed number should be your student ID.
# If it has letter, please ignore it and use only numbers in the ID
np.random.seed(18061077)

# Shipment cost from factory to distribution center
RandomFtoD1 = np.round(np.random.uniform(low = 1, high = 10, size = 12),0).astype(int)
RandomFtoD2 = np.reshape(RandomFtoD1, (4,3))
CostFtoD = pd.DataFrame(RandomFtoD2, columns=['Atlas', 'Nebule',"Azure"],
                        index=["Tonoz","White Kiosk","Quarterage","Middle Yard"])
#CostFtoD # gives the shipment cost from factory to distribution center

# Shipment cost from distribution center to store
RandomDtoS1 = np.round(np.random.uniform(low = 1, high = 10, size = 15),0).astype(int)
RandomDtoS2 = np.reshape(RandomDtoS1, (3,5))
CostDtoS = pd.DataFrame(RandomDtoS2, columns=["Venus", "Minerva", "Neptunus", "Mars", "Ceres"],
                        index=['Atlas', 'Nebule',"Azure"])
#CostDtoS # gives the shipment cost from distribution center to store

# Shipment cost from factory to store
RandomFtoS1 = np.round(np.random.uniform(low = 1, high = 10, size = 20),0).astype(int)
RandomFtoS2 = np.reshape(RandomFtoS1, (4,5))
CostFtoS = pd.DataFrame(RandomFtoS2, columns=["Venus", "Minerva", "Neptunus", "Mars", "Ceres"],
                        index=["Tonoz","White Kiosk","Quarterage","Middle Yard"])
#CostFtoS # gives the shipment cost from distribution center to store

# Your Required Information About Shipment Cost
print("The shipment cost from factory to distribution center")
print(CostFtoD)
print("\n The shipment cost from distribution center to store")
print(CostDtoS)
print("\n The shipment cost from factory to store")
print(CostFtoS)


The shipment cost from factory to distribution center
             Atlas  Nebule  Azure
Tonoz            8       5      4
White Kiosk      7       3      3
Quarterage      10       7      5
Middle Yard      5       3      5

 The shipment cost from distribution center to store
        Venus  Minerva  Neptunus  Mars  Ceres
Atlas       5        6         4     9     10
Nebule      4       10         4     1      4
Azure       2        1         2     8     10

 The shipment cost from factory to store
             Venus  Minerva  Neptunus  Mars  Ceres
Tonoz            9        3         4     3      5
White Kiosk      9        8         8     5      1
Quarterage       4        1        10     7      6
Middle Yard      2        6         9     4     10


# ~ **SOLUTION** ~#

In [ ]:
#---- Downloading and importing neccesarry library ----#
#!pip install pulp
import pulp as p

In [ ]:
#---- Defining Factories, Distribution Centers and Stores ----#
Factories = ["Tonoz", "Qiosk", "Quart", "MYard"]
DCenters = ["Atlas","Nebula","Azure"]
Stores = ["Venus","Minerva","Neptunus","Mars","Ceres"]

In [ ]:
#---- Dictionaries that show the volume of factory and demand of store ----#
volume = {"Tonoz": 3000, "Qiosk": 5000, "Quart": 7500, "MYard":7000}

demand = {"Venus":5500 ,"Minerva":4750 ,"Neptunus":6550 ,"Mars":4000 ,"Ceres":1700}

#---- Cost of shipping Factory to DC ----#
costFtoD = {"Tonoz":{"Atlas":8 ,"Nebula":5 ,"Azure":4 },
            "Qiosk":{"Atlas":7 ,"Nebula":3 ,"Azure":3 },
            "Quart":{"Atlas":10 ,"Nebula":7 ,"Azure":5 },
            "MYard":{"Atlas":5 ,"Nebula":3 ,"Azure":5 }}
#---- Cost of shipping DC to Store ----#
costDtoS = {"Atlas":{"Venus":5 ,"Minerva":6 ,"Neptunus":4 ,"Mars":9 ,"Ceres":10 },
            "Nebula":{"Venus":4 ,"Minerva":10 ,"Neptunus":4 ,"Mars":1 ,"Ceres":4 },
            "Azure":{"Venus":2 ,"Minerva":1 ,"Neptunus":2 ,"Mars":8 ,"Ceres":10 }}
#---- Cost of shipping Factory to Store ----#
costFtoS = {"Tonoz":{"Venus":9 ,"Minerva":3 ,"Neptunus":4 ,"Mars":3 ,"Ceres":5 },
            "Qiosk":{"Venus":9 ,"Minerva":8 ,"Neptunus":8 ,"Mars":5 ,"Ceres":1 },
            "Quart":{"Venus":4 ,"Minerva":1 ,"Neptunus":10 ,"Mars":7 ,"Ceres":6 },
            "MYard":{"Venus":2 ,"Minerva":6 ,"Neptunus":9 ,"Mars":4 ,"Ceres":10 }}

In [ ]:
#---- Setting up the model ----#
model = p.LpProblem("Transportation", p.LpMinimize)

In [ ]:
#---- Defining route matrices ----#
routeFtoD = [(i,j) for i in Factories for j in DCenters]
routeDtoS = [(j,k) for j in DCenters for k in Stores]
routeFtoS = [(i,k) for i in Factories for k in Stores]

In [ ]:
#---- Defining Decision Variables of Each Routes ----#
FtoD_vars = p.LpVariable.dicts("ShipAmount_FtoD",(Factories,DCenters), lowBound=0)
DtoS_vars = p.LpVariable.dicts("ShipAmount_DtoS",(DCenters,Stores), lowBound=0)
FtoS_vars = p.LpVariable.dicts("ShipAmount_FtoS",(Factories,Stores), lowBound=0)


In [ ]:
#---- Defining Objective Function ----#
model += (p.lpSum(FtoD_vars[i][j]*costFtoD[i][j] for (i,j) in routeFtoD)+
          p.lpSum(DtoS_vars[j][k]*costDtoS[j][k] for (j,k) in routeDtoS)+
          p.lpSum(FtoS_vars[i][k]*costFtoS[i][k] for (i,k) in routeFtoS))


In [ ]:
#---- Determining Constraints ----#
for k in Stores:
  model += p.lpSum(FtoS_vars[i][k] for i in Factories) + p.lpSum(DtoS_vars[j][k] for j in DCenters) >= demand[k]

for j in DCenters:
  model += p.lpSum(FtoD_vars[i][j] for i in Factories) >= p.lpSum(DtoS_vars[j][k] for k in Stores)

for i in Factories:
  model += p.lpSum(FtoS_vars[i][k] for k in Stores) + p.lpSum(FtoD_vars[i][j] for j in DCenters) <= volume[i]

In [ ]:
#---- Checking if our model is optimal ----#
model.solve()   # if returns 1, it means that model is optimal

In [ ]:
#---- Checking if our model is optimal ----#
print("Status:", p.LpStatus[model.status])

In [ ]:
#---- Shows the value of the decision variables ----#
for v in model.variables():
  if v.varValue > 0:
    print(v.name, "=", v.varValue)

In [ ]:
#---- Shows the value of objective function ----#
print("Total cost = ", p.value(model.objective))